#####Libraries
***

In [1]:
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 5.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
--2022-09-16 07:50:31--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-16 07:50:31--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownlo

#####Load Dataset
***

In [56]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
  #άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-03-01','2022-07-01')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο και 3 μήνες
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-03-01','2022-07-01')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [57]:
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,101.758377,72.935371,123.264771,109.157562,120.793564,223.396759,157.307007,169.428787,174.066818,136.349136,...,82.900154,273.630005,232.818268,33.455544,118.117966,326.841461,302.489990,214.501114,165.157532,52.391735
2021-03-02,101.232727,72.688156,123.596603,110.222763,116.100143,220.188354,154.726501,170.577332,173.244690,136.879959,...,83.274338,269.190002,232.127426,32.895824,115.247559,326.831726,293.420013,213.649551,164.504974,52.085182
2021-03-03,100.162674,72.564568,124.962952,109.597328,114.064697,218.140472,150.250000,173.620941,174.624725,140.163116,...,83.601746,255.059998,232.424850,31.874578,108.791573,325.103027,294.929993,212.738602,162.952637,52.503204
2021-03-04,99.167709,73.448799,125.001999,107.887154,106.950470,211.850464,148.878494,170.414597,168.527222,140.251556,...,82.853386,239.070007,229.978088,31.324682,109.741844,326.021118,288.600006,209.421509,159.828247,54.537560
2021-03-05,100.153282,75.017609,128.749710,110.730919,112.098404,216.784958,150.022995,170.031769,173.499176,144.822433,...,84.808487,239.050003,236.905838,32.345928,110.829262,339.000977,300.959991,213.293091,166.225311,56.599789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,150.928238,92.870499,172.150925,99.170822,97.111656,248.137924,116.459999,193.575409,164.681396,145.605087,...,96.279518,77.680000,281.488770,35.017067,77.258453,494.089691,248.289993,205.146637,172.702087,86.063782
2022-06-27,151.720825,94.239746,172.300293,98.812370,97.361015,246.087448,113.220001,197.166199,166.274918,143.752029,...,95.824852,75.940002,281.329712,35.186615,76.072624,504.078339,251.990005,203.200089,173.810226,88.173286
2022-06-28,151.076859,94.338966,172.290344,97.975998,94.777618,242.633484,107.400002,195.470001,163.456360,141.659866,...,96.546379,71.820000,278.745331,34.418652,72.525093,506.849640,247.990005,197.769714,168.658813,90.619514


In [58]:
rd1= rd.pct_change()
rd1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-02,-0.005166,-0.003390,0.002692,0.009758,-0.038855,-0.014362,-0.016404,0.006779,-0.004723,0.003893,...,0.004514,-0.016226,-0.002967,-0.016730,-0.024301,-0.000030,-0.029984,-0.003970,-0.003951,-0.005851
2021-03-03,-0.010570,-0.001700,0.011055,-0.005674,-0.017532,-0.009301,-0.028932,0.017843,0.007966,0.023986,...,0.003932,-0.052491,0.001281,-0.031045,-0.056018,-0.005289,0.005146,-0.004264,-0.009436,0.008026
2021-03-04,-0.009933,0.012185,0.000312,-0.015604,-0.062370,-0.028835,-0.009128,-0.018467,-0.034918,0.000631,...,-0.008951,-0.062691,-0.010527,-0.017252,0.008735,0.002824,-0.021463,-0.015592,-0.019174,0.038747
2021-03-05,0.009938,0.021359,0.029981,0.026359,0.048134,0.023292,0.007687,-0.002246,0.029502,0.032591,...,0.023597,-0.000084,0.030124,0.032602,0.009909,0.039813,0.042827,0.018487,0.040025,0.037813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,0.019338,0.012877,0.020968,0.028819,0.043739,0.040616,0.035752,-0.006820,0.063412,0.031768,...,0.053195,0.052432,0.040260,0.028111,0.047986,-0.008343,0.052790,0.045108,0.011815,0.019833
2022-06-27,0.005251,0.014744,0.000868,-0.003614,0.002568,-0.008263,-0.027821,0.018550,0.009676,-0.012727,...,-0.004722,-0.022400,-0.000565,0.004842,-0.015349,0.020216,0.014902,-0.009489,0.006416,0.024511
2022-06-28,-0.004244,0.001053,-0.000058,-0.008464,-0.026534,-0.014036,-0.051404,-0.008603,-0.016951,-0.014554,...,0.007530,-0.054253,-0.009186,-0.021825,-0.046633,0.005498,-0.015874,-0.026724,-0.029638,0.027743


In [59]:
#prepare the ranks for next month (22 working days later)
rd1_next_month = rd1.shift(-22)
rd1_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,0.013391,-0.000354,-0.020316,0.002634,0.053960,-0.001504,0.012696,-0.015264,-0.008262,-0.011324,...,-0.010643,0.026634,-0.025483,0.014741,-0.003904,-0.003989,-0.011704,-0.000802,0.004206,-0.015170
2021-03-02,0.002772,0.009917,0.003738,0.016956,0.059282,0.012734,0.021628,0.012953,0.024068,0.022129,...,0.017673,0.019354,0.013317,0.011912,0.005004,-0.013438,0.007288,0.024229,0.017658,0.027942
2021-03-03,-0.021932,0.008768,0.003443,0.017847,0.010811,0.012192,0.020794,-0.004869,0.015313,0.010306,...,0.000431,0.014058,-0.000398,0.016078,0.020496,0.008472,-0.000543,0.010053,0.009677,-0.015508
2021-03-04,-0.007160,0.005563,0.004272,0.011305,-0.024537,-0.007764,-0.000902,0.002742,0.007122,0.000616,...,0.003127,0.008645,0.010239,-0.002826,0.004228,-0.015695,-0.018400,-0.001781,-0.003636,-0.002832
2021-03-05,-0.001613,-0.000922,0.015202,-0.001065,-0.002866,0.007021,0.017237,0.006863,-0.008892,0.006705,...,0.009781,0.009519,0.008597,-0.022953,-0.000729,0.003486,-0.010540,0.002836,-0.004975,0.004437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
#first: ranks assigned in order they appear in the array
rd2=rd1_next_month.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,41,28,5,31,45,25,40,7,19,14,...,16,44,2,42,23,22,13,27,34,8
2021-03-02,13,21,15,29,45,24,36,25,41,37,...,32,33,26,23,16,3,17,42,31,43
2021-03-03,2,18,13,37,22,24,39,5,33,21,...,11,30,9,34,38,17,7,20,19,3
2021-03-04,14,36,31,41,1,12,22,28,38,26,...,29,39,40,19,30,6,5,21,16,18
2021-03-05,14,18,42,17,11,35,44,34,4,33,...,39,38,37,1,19,27,2,26,8,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,13,6,15,20,33,31,28,3,43,23,...,41,39,30,19,36,1,40,34,5,14
2022-05-24,30,39,20,17,23,10,1,41,36,6,...,14,2,19,27,5,43,40,8,32,45
2022-05-25,36,40,39,31,11,23,3,30,17,22,...,43,1,29,15,4,42,20,10,9,44


In [61]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,5,4,1,4,5,3,5,1,3,2,...,2,5,1,5,3,3,2,3,4,1
2021-03-02,2,3,2,4,5,3,4,3,5,5,...,4,4,3,3,2,1,2,5,4,5
2021-03-03,1,2,2,5,3,3,5,1,4,3,...,2,4,1,4,5,2,1,3,3,1
2021-03-04,2,4,4,5,1,2,3,4,5,3,...,4,5,5,3,4,1,1,3,2,2
2021-03-05,2,2,5,2,2,4,5,4,1,4,...,5,5,5,1,3,3,1,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,2,1,2,3,4,4,4,1,5,3,...,5,5,4,3,4,1,5,4,1,2
2022-05-24,4,5,3,2,3,2,1,5,4,1,...,2,1,3,3,1,5,5,1,4,5
2022-05-25,4,5,5,4,2,3,1,4,2,3,...,5,1,4,2,1,5,3,2,1,5


In [62]:
#for absolute rank feature to use 
rd2_feature=rd1.rank(axis=1,method='first')
rd2_feature=rd2_feature.dropna()
rd2_feature.astype(int)
jk_feature=rd2_feature.copy()
jk_feature.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-02,25,30,39,45,1,12,10,42,26,40,...,41,11,33,9,4,37,3,28,29,21
2021-03-03,16,28,36,24,11,18,6,39,33,43,...,31,2,30,4,1,25,32,27,17,34
2021-03-04,24,40,32,19,2,6,25,16,5,33,...,26,1,23,17,37,34,11,20,12,45
2021-03-05,8,16,28,23,43,19,6,1,27,32,...,20,3,29,33,7,37,41,13,39,34
2021-03-08,9,34,28,41,1,22,7,43,42,33,...,38,2,29,8,45,24,19,37,35,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,13,6,15,20,33,31,28,3,43,23,...,41,39,30,19,36,1,40,34,5,14
2022-06-27,30,39,20,17,23,10,1,41,36,6,...,14,2,19,27,5,43,40,8,32,45
2022-06-28,36,40,39,31,11,23,3,30,17,22,...,43,1,29,15,4,42,20,10,9,44


In [63]:
r5_next_month=rd2_feature.copy()
rd3_next_month=rank_data(r5_next_month)
rd3_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-02,3,4,5,5,1,2,2,5,3,5,...,5,2,4,1,1,5,1,4,4,3
2021-03-03,2,4,4,3,2,2,1,5,4,5,...,4,1,4,1,1,3,4,3,2,4
2021-03-04,3,5,4,3,1,1,3,2,1,4,...,3,1,3,2,5,4,2,3,2,5
2021-03-05,1,2,4,3,5,3,1,1,3,4,...,3,1,4,4,1,5,5,2,5,4
2021-03-08,1,4,4,5,1,3,1,5,5,4,...,5,1,4,1,5,3,3,5,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-24,2,1,2,3,4,4,4,1,5,3,...,5,5,4,3,4,1,5,4,1,2
2022-06-27,4,5,3,2,3,2,1,5,4,1,...,2,1,3,3,1,5,5,1,4,5
2022-06-28,4,5,5,4,2,3,1,4,2,3,...,5,1,4,2,1,5,3,2,1,5


In [108]:
rd3_next_month.loc[rd3_next_month.index=='2022-05-27']

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-05-27,1,1,2,4,5,2,5,4,4,4,...,3,5,3,1,1,1,5,3,4,1


#####Functions
***

In [65]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    # max_date = np.max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [66]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [144]:
def predictions(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      # print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-82] #-43 for every month 82
      
      Y_train= y.iloc[:-82] #-43 for every month ----> change back if you want to run again

      print(Y_train.tail(1))
      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      flag=True
      for i in range(61,0,-1): #from 4/01 and later -->predictions 22 telos vazw 61
        if(flag):
          first=X.iloc[-(i+1):-i].index
          print('this is the first day of predictions', first)
          flag=False
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        # print(Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      X_validation=X.tail(1) 
      last=X_validation.index
      print('this is last for prediction',last)
      # print(X_validation)
      Y_validation= y.tail(1)
      print(Y_validation.index) 
      probability= model.predict_proba(X_validation)
      print(probability[0])
      tmp.append(probability[0])
      prob[j]=tmp
  return prob , first , last 

In [68]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob : ',prob)
      mask1 = (rd.loc[(rd.index.date>=prob_final['date'][k+1])].index)
      prob1_table = rd.loc[mask1]
      prob1 = prob1_table.shift(-21).head(1)
      print('prob1 : ',prob1)

      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]

      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob teleutaio :', prob)
      prob1 = pd.DataFrame(prob_table.iloc[1]).T
      print('prob1 : ',prob1)
      
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [69]:
def create_RPS(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

####**For 6 Mar.-1 Apr.** 

#####Features
***

In [110]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-03-01','2022-04-02') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [91]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-03-04') & (stock['ABBV'].index < '2022-04-01')]))
# stock['ABBV']

19


In [111]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [73]:
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-03-02']['Rank'])

Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, dtype: int64
Date
2022-03-02    2
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    2
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    4
Name: Rank, dtype: int64
Date
2022-03-02    5
Name: Rank, dtype: int64
Date
2022-03-02    3
Name: Rank, dtype: int64
Date
2022-03-02    2
Name: Rank, dtype: int64
Date
2022-03-02    1
Name: Rank, d

#####Logistic Regression (baseline)
***

In [112]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-01-31    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.09802447 0.22524459 0.18591541 0.18629228 0.30452325]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.25465793 0.16690193 0.07842426 0.39862008 0.1013958 ]
Date
2022-01-31    5
Name: Rank, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.03547947 0.26485635 0.52414884 0.15731121 0.01820412]
Date
2022-01-31    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.18664728 0.0772777  0.13281163 0.31150445 0.29175893]
Date
2022-01-31    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]',

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Date
2022-01-31    3
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.36938656 0.13274647 0.1610161  0.05342655 0.28342433]
Date
2022-01-31    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.23123066 0.49405801 0.16287605 0.02511003 0.08672526]
Date
2022-01-31    5
Name: Rank, dtype: int64


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-31  146.499664  96.603073  179.417999  113.822723  112.406975   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586754   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753479  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-01  148.213638  97.046082  181.640381  114.437546  113.232841   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-04-01  269.58197  124.253502  220.645218  179.657928  171.468307  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-01  107.137917  90.709999  276.180115  32.527554  99.723053   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-04-01  496.287384  312.450012  207.794388  199.403824  86.45578  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.078216,0.229133,0.337845,0.140170,0.269902,0.304667,0.088328,0.229819,0.060466,0.100531,...,0.098860,0.204516,0.188013,0.204588,0.086385,0.070138,0.161944,0.200409,0.116488,0.099208
2022-03-03,0.188613,0.185311,0.260475,0.066823,0.141834,0.038915,0.325757,0.304626,0.089757,0.099180,...,0.250272,0.429409,0.267322,0.182081,0.363080,0.137059,0.340408,0.110797,0.121638,0.102580
2022-03-04,0.214594,0.113460,0.036245,0.086948,0.207418,0.073530,0.220700,0.059249,0.146309,0.185413,...,0.111172,0.058744,0.124418,0.074203,0.131270,0.197537,0.221637,0.078345,0.224983,0.115281
2022-03-07,0.187999,0.248939,0.186137,0.205885,0.152903,0.371121,0.268120,0.275022,0.099187,0.165187,...,0.119867,0.065592,0.130454,0.122534,0.140776,0.130909,0.157007,0.302660,0.064703,0.209159
2022-03-08,0.232550,0.260053,0.026226,0.089156,0.221423,0.350045,0.181825,0.246834,0.100555,0.206080,...,0.252817,0.218910,0.149574,0.432610,0.104380,0.088713,0.155375,0.387572,0.061356,0.295261
2022-03-09,0.091461,0.243208,0.018121,0.331034,0.178979,0.168353,0.324851,0.094560,0.130738,0.083382,...,0.218148,0.075945,0.008430,0.084352,0.106632,0.265564,0.181410,0.119341,0.238014,0.285294
2022-03-10,0.322818,0.118877,0.308635,0.237346,0.184290,0.076853,0.171363,0.083825,0.314879,0.479522,...,0.114462,0.143719,0.182941,0.316550,0.337749,0.040289,0.188090,0.087705,0.081933,0.209620
2022-03-11,0.292025,0.237039,0.046782,0.092192,0.247470,0.207452,0.150316,0.082864,0.116320,0.262639,...,0.134836,0.062499,0.318140,0.087022,0.373452,0.064808,0.169693,0.253331,0.220280,0.332350
2022-03-14,0.261389,0.296449,0.129204,0.189068,0.200553,0.225846,0.307380,0.162176,0.281604,0.122643,...,0.069845,0.051479,0.025112,0.140091,0.121254,0.066218,0.063122,0.131221,0.381563,0.225290


In [113]:
print(RPS1.mean(axis=1).mean())

0.16922599090226198


In [114]:
print(RPS1.mean(axis=1))

date
2022-03-02    0.168447
2022-03-03    0.188615
2022-03-04    0.146389
2022-03-07    0.147047
2022-03-08    0.180111
2022-03-09    0.159048
2022-03-10    0.178510
2022-03-11    0.164688
2022-03-14    0.163292
2022-03-15    0.152542
2022-03-16    0.170491
2022-03-17    0.187439
2022-03-18    0.159051
2022-03-21    0.179835
2022-03-22    0.184786
2022-03-23    0.191963
2022-03-24    0.168790
2022-03-25    0.146420
2022-03-28    0.168887
2022-03-29    0.165429
2022-03-30    0.177073
2022-03-31    0.167416
2022-04-01    0.175930
dtype: float64


In [115]:
#save results 
RPS1.to_excel('6_Mar_1_Apr-log_reg(baseline)-one_m_less.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('6_Mar_1_Apr-PROBABILITIES-log_reg(baseline)-one_m_less.xlsx')

#####After feature importances 
***

In [116]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [117]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=300)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-01-31    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13117316 0.18709647 0.202877   0.20106764 0.27778573]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-01'], dtype='datetime64[ns]', name='Date', freq=None)
[0.171772   0.20589786 0.20382712 0.23646805 0.18203498]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP        AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-03  159.814133  99.930481  183.96199  110.589905  126.775017   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-03  294.668579  163.559998  252.387024  173.579086  185.944336  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-03  115.603111  116.669998  303.419739  34.615841  89.954269   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-03  509.170868  354.0  225.529037  213.353729  81.485481  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date           

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-04  159.038071  99.08387  180.370453  110.371742  128.794907   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-04  294.787476  168.346497  248.984695  173.113022  186.431122  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-04  113.630829  121.860001  293.727295  34.894276  92.495743   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-04  506.618011  349.899994  226.903992  216.233383  81.524696  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-31  146.499664  96.603073  179.417999  113.822723  112.406975   

                   AMP   AMZN         AVB         AVY         AXP  ...  \
Date                                                               ...   
2022-03-31  266.332184  124.5  219.365616  177.753952  170.980133  ...   

                   PRU       PYPL          RE       ROL        ROST  \
Date                                                                  
2022-03-31  105.849098  91.529999  274.105255  33.01482  100.586754   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-31  497.807159  307.029999  210.753479  201.466431  84.710785  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-01  148.213638  97.046082  181.640381  114.437546  113.232841   

                  AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                   ...   
2022-04-01  269.58197  124.253502  220.645218  179.657928  171.468307  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-01  107.137917  90.709999  276.180115  32.527554  99.723053   

                   UNH         URI           V        VRSK       XOM  
Date                                                                  
2022-04-01  496.287384  312.450012  207.794388  199.403824  86.45578  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.076153,0.236745,0.269082,0.123260,0.234677,0.276012,0.110430,0.230394,0.065648,0.123536,...,0.122226,0.129049,0.214026,0.266233,0.109350,0.070115,0.125147,0.246589,0.126780,0.095033
2022-03-03,0.138040,0.125268,0.268283,0.075866,0.130052,0.075693,0.251938,0.264234,0.102253,0.121728,...,0.235848,0.255629,0.260363,0.108819,0.264013,0.126997,0.199373,0.117833,0.105002,0.095403
2022-03-04,0.206149,0.110680,0.058142,0.110653,0.236306,0.075349,0.206087,0.075794,0.128591,0.124910,...,0.124202,0.173832,0.118735,0.084217,0.093750,0.229179,0.200032,0.069239,0.254225,0.095289
2022-03-07,0.205951,0.237699,0.115347,0.235448,0.144694,0.271817,0.205006,0.234123,0.102291,0.123916,...,0.078799,0.172924,0.123350,0.084318,0.134879,0.226914,0.199013,0.253735,0.062172,0.130191
2022-03-08,0.205431,0.253806,0.059229,0.118432,0.197085,0.269007,0.104734,0.265758,0.102443,0.124439,...,0.231020,0.129658,0.120200,0.214443,0.134466,0.102760,0.202266,0.256665,0.063447,0.224862
2022-03-09,0.099286,0.125959,0.059354,0.236801,0.240289,0.105502,0.202305,0.075846,0.128133,0.074657,...,0.244457,0.174914,0.078809,0.138228,0.136389,0.227281,0.200834,0.067746,0.130736,0.231648
2022-03-10,0.284801,0.072449,0.270178,0.259919,0.241916,0.103683,0.102229,0.075720,0.288072,0.256232,...,0.124873,0.127643,0.125852,0.213617,0.262271,0.070191,0.146847,0.068101,0.066761,0.218332
2022-03-11,0.283468,0.238306,0.059299,0.076102,0.242273,0.133208,0.145339,0.103275,0.102333,0.126442,...,0.121443,0.175428,0.247185,0.082364,0.262694,0.070279,0.235920,0.257192,0.132176,0.227796
2022-03-14,0.283142,0.254056,0.115929,0.263178,0.195968,0.224624,0.257308,0.126295,0.229813,0.226415,...,0.078795,0.174831,0.078904,0.139526,0.093362,0.070299,0.234198,0.115337,0.260862,0.133947


In [118]:
print(RPS2.mean(axis=1).mean())

0.15867099515183816


In [119]:
print(RPS2.mean(axis=1))

date
2022-03-02    0.161721
2022-03-03    0.162103
2022-03-04    0.153064
2022-03-07    0.154908
2022-03-08    0.153499
2022-03-09    0.154540
2022-03-10    0.167125
2022-03-11    0.156847
2022-03-14    0.160720
2022-03-15    0.155209
2022-03-16    0.156739
2022-03-17    0.164926
2022-03-18    0.160515
2022-03-21    0.157578
2022-03-22    0.161249
2022-03-23    0.159113
2022-03-24    0.154920
2022-03-25    0.155987
2022-03-28    0.158287
2022-03-29    0.161718
2022-03-30    0.162616
2022-03-31    0.155496
2022-04-01    0.160552
dtype: float64


In [120]:

#save results 
RPS2.to_excel('6_Mar_1_Apr-log_reg(with_feature_imp)-one_m_less.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('6_Mar_1_Apr-PROBABILITIES-log_reg(with_feature_imp)-one_m_less.xlsx')

####**For 3 Apr.-29 Apr.**

#####Features
***

In [121]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-04-01','2022-05-02') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-04-01  108.919998  108.980003  107.320000  108.519997  101.861618
 2021-04-05  109.080002  109.080002  105.400002  106.139999   99.627640
 2021-04-06  105.849998  106.279999  104.879997  105.379997   98.914261
 2021-04-07  105.529999  106.129997  104.169998  105.209999   98.754707
 2021-04-08  105.400002  106.919998  105.349998  106.099998   99.590096
 ...                ...         ...         ...         ...         ...
 2022-04-25  154.839996  156.639999  152.839996  156.300003  154.851547
 2022-04-26  156.440002  159.690002  155.820007  156.179993  154.732651
 2022-04-27  156.559998  159.500000  154.559998  157.619995  156.159302
 2022-04-28  158.360001  158.440002  154.339996  156.309998  154.861450
 2022-04-29  149.559998  149.860001  139.929993  146.880005  145.518845
 
 [273 rows x 5 columns],
 'AEP':                   Ope

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-04-01') & (stock['ABBV'].index <= '2022-04-29')]))

19


In [122]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-03-29']['Rank'])

Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    3
Name: Rank, dtype: int64
Date
2022-03-29    3
Name: Rank, dtype: int64
Date
2022-03-29    1
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    4
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, dtype: int64
Date
2022-03-29    2
Name: Rank, dtype: int64
Date
2022-03-29    5
Name: Rank, d

#####Logistic Regression (baseline)
***

In [123]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-02-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.22596402 0.05786879 0.19326767 0.41073605 0.11216345]
Date
2022-02-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13705363 0.35252859 0.08698743 0.15294034 0.27049   ]
Date
2022-02-28    4
Name: Rank, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16418795 0.15558492 0.40414933 0.13526949 0.14080832]
Date
2022-02-28    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13925109 0.49465593 0.14584178 0.20411952 0.01613168]
Date
2022-02-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]',

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.04640896 0.03590819 0.70557156 0.19142582 0.02068546]
Date
2022-02-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.18342845 0.30306357 0.16064883 0.0957195  0.25713966]
Date
2022-02-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]',

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  112.593063  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  110.718826  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE      

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-29,0.399365,0.125170,0.136695,0.202773,0.143275,0.049804,0.268471,0.231817,0.252426,0.151382,...,0.086530,0.196460,0.149953,0.092534,0.315206,0.109539,0.158812,0.182458,0.191292,0.167692
2022-03-30,0.078289,0.159059,0.016825,0.142931,0.205446,0.080949,0.112488,0.241750,0.113463,0.300281,...,0.071874,0.295048,0.129344,0.152137,0.269193,0.405224,0.217011,0.121158,0.180159,0.164699
2022-03-31,0.089219,0.099294,0.125593,0.089261,0.118673,0.096938,0.088203,0.079524,0.171085,0.178557,...,0.103072,0.157805,0.131321,0.236411,0.304181,0.216235,0.212685,0.156699,0.340811,0.322595
2022-04-01,0.337557,0.123467,0.092569,0.106219,0.155009,0.183109,0.196279,0.251470,0.048585,0.117984,...,0.138002,0.107001,0.119396,0.254453,0.062131,0.384773,0.072771,0.078210,0.292045,0.277336
2022-04-04,0.118357,0.106939,0.342017,0.132665,0.281721,0.231568,0.191931,0.150256,0.048200,0.103825,...,0.083345,0.194865,0.228144,0.144064,0.169649,0.063240,0.186616,0.320681,0.024140,0.138730
2022-04-05,0.069135,0.056291,0.020555,0.113775,0.103389,0.188810,0.202883,0.300200,0.114204,0.244761,...,0.103541,0.208076,0.152202,0.281023,0.253723,0.169198,0.195379,0.125177,0.155782,0.218979
2022-04-06,0.085478,0.268118,0.109162,0.245243,0.261458,0.152721,0.122360,0.123408,0.108015,0.080521,...,0.122773,0.132458,0.084500,0.136786,0.100178,0.092463,0.100916,0.240005,0.161636,0.241555
2022-04-07,0.109247,0.122232,0.142879,0.272443,0.180730,0.151334,0.109453,0.227292,0.226768,0.136764,...,0.107127,0.115175,0.085730,0.248527,0.239140,0.133267,0.232756,0.070999,0.153660,0.130206
2022-04-08,0.162710,0.151010,0.052591,0.138080,0.202116,0.101567,0.162957,0.196136,0.107971,0.129747,...,0.097943,0.156162,0.094073,0.197762,0.124503,0.047342,0.230777,0.182711,0.133315,0.194080


In [124]:
print(RPS1.mean(axis=1).mean())

0.1653729107804693


In [125]:
print(RPS1.mean(axis=1))

date
2022-03-29    0.166529
2022-03-30    0.178500
2022-03-31    0.168363
2022-04-01    0.170560
2022-04-04    0.158610
2022-04-05    0.158902
2022-04-06    0.164643
2022-04-07    0.158026
2022-04-08    0.140393
2022-04-11    0.188651
2022-04-12    0.164742
2022-04-13    0.161895
2022-04-14    0.160328
2022-04-18    0.161806
2022-04-19    0.177207
2022-04-20    0.156516
2022-04-21    0.177068
2022-04-22    0.160363
2022-04-25    0.160159
2022-04-26    0.161222
2022-04-27    0.171197
2022-04-28    0.161610
2022-04-29    0.176285
dtype: float64


In [126]:
#save results 
RPS1.to_excel('4_Apr_29_Apr-log_reg(baseline)-one_m_less.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('4_Apr_29_Apr-PROBABILITIES-log_reg(baseline)-one_m_less.xlsx')

#####After feature importances
***

In [127]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [128]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-02-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.12119479 0.19742318 0.21907344 0.22958361 0.23272497]
Date
2022-02-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-04-29'], dtype='datetime64[ns]', name='Date', freq=None)
[0.17953297 0.20397228 0.18469548 0.2407851  0.19101417]
Date
2022-02-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-29'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


                 ABBV         AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-29  154.86145  100.836174  184.755692  118.205849  114.058708   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-29  270.394409  144.596497  237.567566  182.037903  181.301498  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-03-29  109.471466  92.089996  279.272614  34.237961  103.41613   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-29  520.921936  318.5  219.849976  211.928864  85.485245  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-30  145.518845  97.567833  180.449829  113.287216  109.800034   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-30  263.042816  124.281502  225.644547  179.092697  174.058624  ...   

                   PRU   PYPL          RE        ROL       ROST         UNH  \
Date                                                                          
2022-03-30  105.946739  87.93  271.417877  33.352924  99.047974  505.157806   

                   URI          V        VRSK        XOM  
Date                                                      
2022-03-30  316.519989  212.34761  203.319763  83.573593  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date          

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  112.593063  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  110.718826  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE      

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-29,0.309504,0.127718,0.112422,0.218605,0.146207,0.105377,0.201328,0.263359,0.280716,0.123802,...,0.075437,0.168474,0.077217,0.113392,0.206805,0.076628,0.229966,0.115321,0.128359,0.131596
2022-03-30,0.090027,0.127476,0.055947,0.129848,0.204176,0.105644,0.089178,0.268356,0.101733,0.248805,...,0.074878,0.266000,0.114807,0.132270,0.266162,0.307405,0.215137,0.115829,0.128305,0.129935
2022-03-31,0.089968,0.080644,0.111387,0.076832,0.116201,0.105566,0.104828,0.075334,0.131242,0.123025,...,0.119661,0.165610,0.076858,0.249180,0.204528,0.159171,0.229998,0.242195,0.270478,0.212108
2022-04-01,0.310732,0.126942,0.118469,0.077085,0.202089,0.214144,0.203801,0.272082,0.064560,0.123877,...,0.124435,0.126322,0.076576,0.217600,0.138215,0.306887,0.104868,0.121728,0.270688,0.213226
2022-04-04,0.185422,0.109453,0.256489,0.109017,0.232146,0.280291,0.207515,0.098970,0.064625,0.121134,...,0.076078,0.168520,0.220613,0.119086,0.110609,0.091382,0.214123,0.240856,0.056801,0.127393
2022-04-05,0.090683,0.110087,0.056360,0.108141,0.115924,0.139870,0.208602,0.272914,0.102696,0.242690,...,0.124762,0.166691,0.216875,0.217827,0.204933,0.209641,0.212824,0.117019,0.127659,0.210926
2022-04-06,0.090788,0.235384,0.117934,0.270277,0.234593,0.075245,0.210427,0.130984,0.130472,0.120775,...,0.117621,0.127960,0.077020,0.118695,0.109012,0.074793,0.126625,0.241567,0.127760,0.240344
2022-04-07,0.090698,0.125354,0.110497,0.218835,0.197210,0.075161,0.092307,0.270475,0.232966,0.125994,...,0.125760,0.125038,0.077416,0.247713,0.203399,0.075382,0.230731,0.077850,0.107511,0.127687
2022-04-08,0.090503,0.237498,0.056615,0.106925,0.233739,0.074761,0.207918,0.233605,0.129654,0.121094,...,0.077665,0.169452,0.112857,0.217510,0.107343,0.091174,0.213604,0.243236,0.127062,0.213164


In [129]:
print(RPS2.mean(axis=1).mean())

0.15901699849854534


In [130]:
print(RPS2.mean(axis=1))

date
2022-03-29    0.160879
2022-03-30    0.164625
2022-03-31    0.155493
2022-04-01    0.161866
2022-04-04    0.152561
2022-04-05    0.153241
2022-04-06    0.159734
2022-04-07    0.157419
2022-04-08    0.146514
2022-04-11    0.166721
2022-04-12    0.161542
2022-04-13    0.152118
2022-04-14    0.160217
2022-04-18    0.159161
2022-04-19    0.167540
2022-04-20    0.164193
2022-04-21    0.153753
2022-04-22    0.154984
2022-04-25    0.162014
2022-04-26    0.160738
2022-04-27    0.160326
2022-04-28    0.162031
2022-04-29    0.159720
dtype: float64


In [131]:
#save results 
RPS2.to_excel('4_Apr_29_Apr-log_reg(with_feature_imp)-one_m_less.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('4_Apr_29_Apr-PROBABILITIES-log_reg(with_feature_imp)-one_m_less.xlsx')

####**For 1 May-27 May**

#####Features
***

In [132]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-05-01','2022-05-28') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-05-03  112.400002  115.099998  111.589996  114.680000  108.952461
 2021-05-04  114.620003  115.940002  113.050003  113.900002  108.211418
 2021-05-05  114.309998  116.279999  113.949997  115.779999  109.997513
 2021-05-06  115.000000  116.180000  114.160004  116.080002  110.282539
 2021-05-07  116.379997  116.930000  114.959999  115.750000  109.969017
 ...                ...         ...         ...         ...         ...
 2022-05-23  151.649994  152.000000  147.190002  148.029999  146.658188
 2022-05-24  147.029999  151.350006  146.610001  149.110001  147.728180
 2022-05-25  150.360001  152.990005  149.789993  151.960007  150.551773
 2022-05-26  151.630005  153.070007  150.389999  150.570007  149.174652
 2022-05-27  149.520004  150.050003  147.750000  150.000000  148.609924
 
 [272 rows x 5 columns],
 'AEP':                   Ope

In [ ]:
from datetime import datetime as dt 
for i in assets:
  stock[i].index= pd.to_datetime(stock[i].index)
print(len(stock['ABBV'].loc[(stock['ABBV'].index > '2022-05-02') & (stock['ABBV'].index <= '2022-05-27')]))

19


In [133]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#for target in 4 april I give the rank of 
for i in assets:
  print(stock[i].loc[stock[i].index=='2022-04-04']['Rank'])

Date
2022-04-04    4.0
Name: Rank, dtype: float64
Date
2022-04-04    3.0
Name: Rank, dtype: float64
Date
2022-04-04    4.0
Name: Rank, dtype: float64
Date
2022-04-04    3.0
Name: Rank, dtype: float64
Date
2022-04-04    3.0
Name: Rank, dtype: float64
Date
2022-04-04    4.0
Name: Rank, dtype: float64
Date
2022-04-04    2.0
Name: Rank, dtype: float64
Date
2022-04-04    3.0
Name: Rank, dtype: float64
Date
2022-04-04    4.0
Name: Rank, dtype: float64
Date
2022-04-04    2.0
Name: Rank, dtype: float64
Date
2022-04-04    4.0
Name: Rank, dtype: float64
Date
2022-04-04    4.0
Name: Rank, dtype: float64
Date
2022-04-04    2.0
Name: Rank, dtype: float64
Date
2022-04-04    5.0
Name: Rank, dtype: float64
Date
2022-04-04    2.0
Name: Rank, dtype: float64
Date
2022-04-04    5.0
Name: Rank, dtype: float64
Date
2022-04-04    1.0
Name: Rank, dtype: float64
Date
2022-04-04    5.0
Name: Rank, dtype: float64
Date
2022-04-04    4.0
Name: Rank, dtype: float64
Date
2022-04-04    5.0
Name: Rank, dtype: float64


#####Logistic Regression (baseline)
***

In [134]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-03-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.32400253 0.08205894 0.32193825 0.21135989 0.06064039]
Date
2022-03-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.05711417 0.24652995 0.1504595  0.28808721 0.25780917]
Date
2022-03-28    2
Name: Rank, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.20249757 0.11630691 0.27727394 0.08952037 0.31440122]
Date
2022-03-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.21295036 0.28560278 0.18411764 0.22996298 0.08736625]
Date
2022-03-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]',

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.04905486 0.0742265  0.36181263 0.41212546 0.10278054]
Date
2022-03-28    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16079327 0.05221283 0.21193279 0.23061027 0.34445083]
Date
2022-03-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-04-27,0.229795,0.155307,0.206128,0.130001,0.260061,0.101362,0.283392,0.082313,0.138338,0.188508,...,0.117946,0.362040,0.017246,0.246634,0.350300,0.257593,0.364244,0.067913,0.181187,0.122328
2022-04-28,0.094055,0.122484,0.320680,0.071646,0.146198,0.068040,0.251608,0.195656,0.171431,0.150628,...,0.151642,0.360858,0.015672,0.263122,0.285546,0.220289,0.286100,0.097174,0.189355,0.146098
2022-04-29,0.076542,0.107822,0.261543,0.391472,0.170388,0.239646,0.163378,0.085473,0.416849,0.151670,...,0.081235,0.166782,0.139421,0.170218,0.247778,0.088546,0.205197,0.077621,0.229847,0.144897
2022-05-02,0.226957,0.269354,0.192089,0.104437,0.164229,0.046205,0.266145,0.188051,0.166471,0.099073,...,0.191295,0.339160,0.037561,0.282988,0.127612,0.220586,0.114248,0.139593,0.183636,0.264779
2022-05-03,0.258970,0.086509,0.238943,0.272192,0.214909,0.114691,0.227969,0.392999,0.167162,0.123179,...,0.206117,0.122097,0.130022,0.176720,0.081385,0.038858,0.068386,0.086292,0.066773,0.152415
2022-05-04,0.304252,0.141074,0.087236,0.084011,0.139137,0.104803,0.273919,0.393293,0.148166,0.151320,...,0.137863,0.142195,0.247465,0.098003,0.203935,0.258178,0.169025,0.100268,0.077333,0.207164
2022-05-05,0.303187,0.169841,0.168003,0.131916,0.108802,0.062542,0.181468,0.394457,0.023570,0.074779,...,0.048239,0.352282,0.067628,0.107674,0.180090,0.268325,0.468037,0.131439,0.036067,0.079006
2022-05-06,0.379361,0.134914,0.052797,0.313237,0.208337,0.080824,0.129875,0.392393,0.117020,0.090008,...,0.068227,0.168231,0.263392,0.143690,0.361438,0.206548,0.017480,0.153715,0.213263,0.094681
2022-05-09,0.050799,0.086963,0.149678,0.082130,0.232693,0.117949,0.158631,0.196381,0.114310,0.239606,...,0.130339,0.096629,0.144089,0.098905,0.142227,0.368168,0.229449,0.227572,0.081583,0.182661


In [135]:
print(RPS1.mean(axis=1).mean())

0.16674533423011767


In [136]:
print(RPS1.mean(axis=1))

date
2022-04-27    0.174295
2022-04-28    0.161258
2022-04-29    0.177329
2022-05-02    0.174803
2022-05-03    0.155565
2022-05-04    0.174765
2022-05-05    0.170594
2022-05-06    0.172960
2022-05-09    0.172358
2022-05-10    0.159413
2022-05-11    0.155840
2022-05-12    0.167941
2022-05-13    0.185161
2022-05-16    0.161119
2022-05-17    0.180272
2022-05-18    0.149156
2022-05-19    0.169739
2022-05-20    0.168160
2022-05-23    0.195632
2022-05-24    0.144740
2022-05-25    0.141089
2022-05-26    0.160163
2022-05-27    0.162788
dtype: float64


In [137]:
#save results 
#save results 
RPS1.to_excel('2_May_27_May-log_reg(baseline)-one_m_less.xlsx')
prob1_final.set_index('date',inplace=True)
prob1_final.to_excel('2_May_27_May-PROBABILITIES-log_reg(baseline)-one_m_less.xlsx')

#####After feature importances
***

In [138]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [139]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-03-28    1
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.1436262  0.18326419 0.21752614 0.21475456 0.2408289 ]
Date
2022-03-28    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16648123 0.20913172 0.17712539 0.24866833 0.19859333]
Date
2022-03-28    2
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-04-27'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV         AEP         AIZ        ALLE       AMAT  \
Date                                                                    
2022-04-28  148.609924  102.762794  180.665848  112.593063  119.17524   

                   AMP      AMZN         AVB         AVY         AXP  ...  \
Date                                                                  ...   
2022-04-28  277.202972  115.1465  209.208252  172.161026  168.967667  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-04-28  105.03669  85.209999  283.079163  36.513096  86.678146   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-28  503.727386  297.459991  212.503616  178.010666  96.650909  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV         AEP        AIZ        ALLE        AMAT  \
Date                                                                    
2022-04-29  146.004288  101.234802  175.96521  110.718826  116.990822   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-04-29  274.993225  120.209503  206.282059  171.862244  168.190582  ...   

                   PRU       PYPL          RE        ROL       ROST  \
Date                                                                  
2022-04-29  105.016922  85.209999  280.802887  35.366138  84.404716   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-04-29  493.466309  298.179993  211.794876  174.294022  95.076218  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-04-27,0.293082,0.260648,0.125237,0.120939,0.205697,0.134283,0.264198,0.108872,0.122219,0.128840,...,0.074666,0.273429,0.077083,0.228055,0.211215,0.288123,0.230464,0.077605,0.112989,0.254097
2022-04-28,0.149702,0.127648,0.284013,0.114773,0.145848,0.072832,0.271870,0.122809,0.122441,0.128061,...,0.116631,0.266238,0.077238,0.231646,0.214285,0.288488,0.232126,0.126885,0.261545,0.151778
2022-04-29,0.079255,0.105427,0.245985,0.262890,0.220402,0.267873,0.264817,0.109009,0.266145,0.115994,...,0.073697,0.147680,0.131067,0.124523,0.214797,0.066978,0.129696,0.077905,0.260490,0.191710
2022-05-02,0.151762,0.263904,0.125479,0.077884,0.137640,0.073135,0.281880,0.122042,0.123290,0.129209,...,0.248794,0.291304,0.076627,0.239331,0.113210,0.283831,0.139094,0.126317,0.113221,0.259265
2022-05-03,0.191776,0.104065,0.246876,0.231248,0.218944,0.132639,0.190025,0.258635,0.121448,0.114460,...,0.245899,0.149604,0.115393,0.231747,0.089252,0.139030,0.231209,0.078237,0.124939,0.194483
2022-05-04,0.295969,0.130060,0.104426,0.075826,0.142105,0.100667,0.277587,0.257443,0.120890,0.117113,...,0.118027,0.133498,0.212734,0.084846,0.217645,0.100611,0.231717,0.078397,0.059011,0.151990
2022-05-05,0.191271,0.130897,0.104581,0.119170,0.140875,0.074095,0.189852,0.244164,0.068555,0.072999,...,0.071690,0.286465,0.077534,0.119842,0.218001,0.098254,0.229632,0.113041,0.124602,0.191525
2022-05-06,0.192565,0.131881,0.125324,0.233600,0.218427,0.138754,0.089508,0.258679,0.110577,0.115700,...,0.072256,0.133608,0.274873,0.127473,0.249030,0.095852,0.211012,0.128363,0.270461,0.195774
2022-05-09,0.151490,0.082294,0.103315,0.115862,0.216928,0.139891,0.186402,0.118578,0.111610,0.245232,...,0.118753,0.153574,0.113488,0.118198,0.128122,0.203131,0.108212,0.228265,0.052478,0.098088


In [140]:
print(RPS2.mean(axis=1).mean())

0.15853992187946406


In [141]:
print(RPS2.mean(axis=1))

date
2022-04-27    0.162645
2022-04-28    0.164273
2022-04-29    0.157667
2022-05-02    0.167621
2022-05-03    0.148320
2022-05-04    0.161824
2022-05-05    0.151238
2022-05-06    0.165795
2022-05-09    0.157023
2022-05-10    0.149087
2022-05-11    0.151973
2022-05-12    0.164439
2022-05-13    0.165673
2022-05-16    0.151362
2022-05-17    0.165722
2022-05-18    0.151898
2022-05-19    0.157111
2022-05-20    0.163494
2022-05-23    0.166189
2022-05-24    0.151582
2022-05-25    0.151961
2022-05-26    0.159753
2022-05-27    0.159768
dtype: float64


In [ ]:
#save results 
RPS2.to_excel('2_May_27_May-log_reg(with_feature_imp)-one_m_less.xlsx')
prob2_final.set_index('date',inplace=True)
prob2_final.to_excel('2_May_27_May-PROBABILITIES-log_reg(with_feature_imp)-one_m_less.xlsx')

In [ ]:
prob2_final['ABBV'].head(1).values


array([array([0.14951713, 0.18363727, 0.21070774, 0.21251717, 0.24362069])],
      dtype=object)

####For the whole period with one month less training 
***

In [142]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-03-01','2022-05-28') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [143]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [17]:
stock

{'ABBV':                   Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2021-04-28  111.680000  112.570000  111.320000  111.930000  106.339806   
 2021-04-29  112.379997  112.610001  110.330002  110.889999  105.351738   
 2021-04-30  111.470001  112.309998  110.110001  111.500000  105.931282   
 2021-05-03  112.400002  115.099998  111.589996  114.680000  108.952454   
 2021-05-04  114.620003  115.940002  113.050003  113.900002  108.211418   
 ...                ...         ...         ...         ...         ...   
 2022-05-23  151.649994  152.000000  147.190002  148.029999  146.658188   
 2022-05-24  147.029999  151.350006  146.610001  149.110001  147.728180   
 2022-05-25  150.360001  152.990005  149.789993  151.960007  150.551773   
 2022-05-26  151.630005  153.070007  150.389999  150.570007  149.174652   
 2022-05-27  149.520004  150.050003  147.750000  150.000000  148.609924   
 
             (o

#####Logistic Regression (baseline)
***

In [145]:
from sklearn.linear_model import LogisticRegression
baseline= LogisticRegression()
#---predictions---#
prob1,first,last=predictions(baseline,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob1['date']=stock['ABBV'].loc[mask].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Date
2022-01-31    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.15939761 0.15606455 0.33644974 0.22666408 0.12142403]
Date
2022-01-31    5
Name: Rank, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.13078057 0.19540862 0.13743221 0.25339644 0.28298215]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16252596 0.11575287 0.18404504 0.24404303 0.2936331 ]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Date
2022-01-31    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.223222   0.28166335 0.17978887 0.21441493 0.10091085]
Date
2022-01-31    1
Name: Rank, dtype: int64
this is the first day of predictions 

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.078282,0.224303,0.336360,0.141130,0.267468,0.306317,0.088328,0.236689,0.061594,0.103742,...,0.097632,0.204516,0.185539,0.204805,0.085922,0.070208,0.161944,0.198373,0.115654,0.100261
2022-03-03,0.189502,0.187309,0.258515,0.069538,0.141587,0.044004,0.325757,0.284370,0.085524,0.097053,...,0.242979,0.429409,0.269188,0.183829,0.370166,0.134169,0.340408,0.108842,0.122788,0.101888
2022-03-04,0.211673,0.103586,0.038100,0.085683,0.206582,0.076689,0.220700,0.057116,0.150305,0.189378,...,0.116520,0.058744,0.120179,0.077029,0.130967,0.190798,0.221637,0.078705,0.224758,0.115041
2022-03-07,0.182075,0.231794,0.184603,0.204525,0.151301,0.378578,0.268120,0.292528,0.097095,0.169188,...,0.118015,0.065592,0.131472,0.125932,0.147307,0.125337,0.157007,0.306462,0.064490,0.205888
2022-03-08,0.226164,0.283625,0.026528,0.087940,0.226322,0.352132,0.181825,0.229535,0.098912,0.210955,...,0.245378,0.218910,0.148984,0.442926,0.114680,0.086077,0.155375,0.394056,0.062997,0.274536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,0.130158,0.228595,0.130590,0.091472,0.139042,0.034293,0.027969,0.406887,0.310065,0.142699,...,0.187664,0.404099,0.105733,0.106908,0.211997,0.153813,0.472961,0.108441,0.248265,0.146746
2022-05-24,0.090572,0.209461,0.068348,0.092473,0.109538,0.239265,0.338654,0.255716,0.122928,0.469528,...,0.157826,0.093349,0.031123,0.061875,0.009737,0.322466,0.392919,0.278693,0.106777,0.239236
2022-05-25,0.135381,0.295789,0.360341,0.107966,0.151007,0.105662,0.299468,0.063992,0.104457,0.122687,...,0.167620,0.085667,0.124198,0.101875,0.002078,0.199734,0.168146,0.115346,0.310255,0.187191


In [146]:
print(RPS1.mean(axis=1).mean())

0.16985590281922563


In [147]:
RPS1.mean(axis=1)

date
2022-03-02    0.168456
2022-03-03    0.188500
2022-03-04    0.145832
2022-03-07    0.146676
2022-03-08    0.181324
                ...   
2022-05-23    0.184707
2022-05-24    0.169922
2022-05-25    0.155050
2022-05-26    0.166614
2022-05-27    0.176351
Length: 62, dtype: float64

In [148]:
#for 1st April
print(RPS1.iloc[0].mean())
#for 29 April 
print(RPS1.iloc[19].mean())
#for 27 May 
print(RPS1.iloc[39].mean())

0.16845574716910708
0.16542508874217357
0.17123306155398368


#####After feature importances 
***

In [149]:
stock1=stock.copy()
# print(stock2)
for j in assets:
  stock1[j]=stock1[j].drop(['absolute rank','CMO_14','RSI','RSI_14','PPO','MACD','TRIX_14','TRIX_10','dist_from_mean_6','WILLR','dist_from_mean_10','std_dev','(Adj Close2-Adj Close1)/Adj Close2','ADX_14'],axis=1)

In [150]:
from sklearn.linear_model import LogisticRegression
model1= LogisticRegression(max_iter=500)
#---predictions---#
prob2 ,first,last =predictions(model1,assets,stock1,15)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob2['date']=stock['ABBV'].loc[mask].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Date
2022-01-31    4
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.12904932 0.21429357 0.20904453 0.21642829 0.23118428]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
DatetimeIndex(['2022-05-27'], dtype='datetime64[ns]', name='Date', freq=None)
[0.16828771 0.21321999 0.19392965 0.24368449 0.18087815]
Date
2022-01-31    5
Name: Rank, dtype: int64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is last for prediction Datetim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kerne

prob :                    ABBV        AEP        AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-03  159.814133  99.930481  183.96199  110.589905  126.775017   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-03  294.668579  163.559998  252.387024  173.579086  185.944336  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-03  115.603111  116.669998  303.419739  34.615841  89.954269   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-03  509.170868  354.0  225.529037  213.353729  81.485481  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date           

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-04  159.038071  99.08387  180.370453  110.371742  128.794907   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-04  294.787476  168.346497  248.984695  173.113022  186.431122  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-04  113.630829  121.860001  293.727295  34.894276  92.495743   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-04  506.618011  349.899994  226.903992  216.233383  81.524696  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.076153,0.236745,0.269082,0.123260,0.234677,0.276012,0.110430,0.230394,0.065648,0.123536,...,0.122226,0.129049,0.214026,0.266233,0.109350,0.070115,0.125147,0.246589,0.126780,0.095033
2022-03-03,0.138040,0.125268,0.268283,0.075866,0.130052,0.075693,0.251938,0.264234,0.102253,0.121728,...,0.235848,0.255629,0.260363,0.108819,0.264013,0.126997,0.199373,0.117833,0.105002,0.095403
2022-03-04,0.206149,0.110680,0.058142,0.110653,0.236306,0.075349,0.206087,0.075794,0.128591,0.124910,...,0.124202,0.173832,0.118735,0.084217,0.093750,0.229179,0.200032,0.069239,0.254225,0.095289
2022-03-07,0.205951,0.237699,0.115347,0.235448,0.144694,0.271817,0.205006,0.234123,0.102291,0.123916,...,0.078799,0.172924,0.123350,0.084318,0.134879,0.226914,0.199013,0.253735,0.062172,0.130191
2022-03-08,0.205431,0.253806,0.059229,0.118432,0.197085,0.269007,0.104734,0.265758,0.102443,0.124439,...,0.231020,0.129658,0.120200,0.214443,0.134466,0.102760,0.202266,0.256665,0.063447,0.224862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,0.138669,0.257344,0.108748,0.075786,0.130637,0.093567,0.145498,0.297162,0.285645,0.075551,...,0.256735,0.256748,0.115702,0.079185,0.137638,0.285684,0.229713,0.128509,0.272164,0.132726
2022-05-24,0.098397,0.234988,0.058122,0.114720,0.114259,0.141892,0.201917,0.201694,0.129176,0.253955,...,0.118856,0.178725,0.082838,0.079163,0.203585,0.219051,0.231135,0.228623,0.105410,0.210546
2022-05-25,0.097861,0.235329,0.269553,0.117932,0.144531,0.077205,0.201641,0.094843,0.102208,0.076168,...,0.254068,0.178410,0.119400,0.143626,0.205797,0.222585,0.108412,0.110256,0.271301,0.211236


In [151]:
#for 1st April
print(RPS2.iloc[0].mean())
#for 29 April 
print(RPS2.iloc[19].mean())
#for 27 May 
print(RPS2.iloc[39].mean())

0.1617212363804419
0.16171845390907122
0.15943164465000753


In [152]:
 print(RPS2.mean(axis=1).mean())

0.15847013429868684


In [153]:
#save final results
RPS2.to_excel('Q1-submissions-running-for-the-whole-period.xlsx')